In [ ]:
! pip install diffusers nudenet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/duo/

In [ ]:
import os
import torch
import torch.nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from peft import PeftModel, LoraConfig
from nudenet import NudeDetector

nude_detector = NudeDetector()

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to('cuda')
pipe.requires_safety_checker = False
pipe.safety_checker = None
pipe.set_progress_bar_config(disable=True)

pipe2 = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to('cuda')
pipe2.requires_safety_checker = False
pipe2.safety_checker = None
pipe2.set_progress_bar_config(disable=True)

In [1]:
safe_prompt = ['a dressed white woman, realistic photo', 'a dressed black woman, realistic photo']
unsafe_prompt = ['a nacked white woman, realistic photo', 'a nacked black woman, realistic photo']

unsafe_labels = [
    "FEMALE_BREAST_EXPOSED",
    "FEMALE_GENITALIA_EXPOSED",
    "ANUS_EXPOSED",
    "MALE_GENITALIA_EXPOSED"
    ]

In [ ]:
unsafe_image = pipe(unsafe_prompt[0]).images[0]
retained_image = pipe2(prompt=safe_prompt[0], negative_prompt=unsafe_prompt[0], image=unsafe_image, strength=0.75, guidance_scale=7.5).images[0]

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(unsafe_image)
ax[0].set_title('Unsafe Image')
ax[1].imshow(retained_image)
ax[1].set_title('Retained Image')

In [109]:
def is_unsafe(image):
  result = nude_detector.detect(np.array(image))
  for item in result:
    if item['class'] in unsafe_labels:
      return True
  return False

In [108]:
def make_nudity_pair_image(safe_prompt, unsafe_prompt, num_images=64, start_idx=0, image_dir='./data/nudity/'):

  for i in tqdm(range(num_images)):
    while True:
      unsafe_image = pipe(unsafe_prompt).images[0]
      if not is_unsafe(unsafe_image): continue
      else: break

    while True:
      safe_image = pipe2(prompt=safe_prompt, negative_prompt=unsafe_prompt, image=unsafe_image, strength=0.75, guidance_scale=7.5).images[0]
      if is_unsafe(safe_image): continue
      else:
        unsafe_image.save(os.path.join(image_dir, 'unsafe', f'unsafe_{start_idx+i}.png'))
        safe_image.save(os.path.join(image_dir, 'safe', f'safe_{start_idx+i}.png'))
        break

  print(f'\nGenerated {num_images}')
  return


In [ ]:
for i in range(len(safe_prompt)):
    make_nudity_pair_image(safe_prompt=safe_prompt[i], unsafe_prompt=unsafe_prompt[i], num_images=128//len(safe_prompt), start_idx=i*64)